In [1]:
import gym
import jax
import tax
import tree
import optax
import jax.numpy as jnp
import numpy as np
import haiku as hk

from a2c.a2c import Data
from a2c.a2c import State
from a2c.a2c import Batch
from a2c.a2c import process_data
from a2c.a2c import loss_fn
from a2c.common.nn import mlp_categorical
from a2c.common.nn import mlp_deterministic
from jax import jit
from jax import vmap
from gym.vector import AsyncVectorEnv
from functools import partial

tax.set_platform('cpu')
rng = jax.random.PRNGKey(42)


def evaluation(rng, env, policy, niters: int = 5):
    all_scores = []
    for _ in range(niters):
        observation, score = env.reset(), 0
        for _ in range(env.spec.max_episode_steps):
            rng, rng_action = jax.random.split(rng)
            action = policy(rng, observation)
            action = int(action)
            observation, reward, done, info = env.step(action)
            score += reward
            if done: 
                break
        all_scores.append(score)
    info = {}
    info['eval/score'] = np.mean(all_scores)
    info['eval/score_std'] = np.std(all_scores)
    return info


@partial(jit, static_argnums=(2, 3, 4))
def update_fn(state, inputs, policy_opt, value_opt, loss_fn):
    """ Generic Update function """
    g, metrics = jax.grad(loss_fn, has_aux=True)(state.params, inputs)
    
    # -- Value Update
    updates, value_opt_state = value_opt(g['value'], state.opt_state['value'])
    value_params = jax.tree_multimap(lambda p, u: p + u, state.params['value'], updates)
    # -- Policy Update
    updates, policy_opt_state = policy_opt(g['policy'], state.opt_state['policy'])
    policy_params = jax.tree_multimap(lambda p, u: p + u, state.params['policy'], updates)
    
    params = state.params
    params = dict(policy=policy_params, value=value_params)
    opt_state = state.opt_state
    opt_state = dict(policy=policy_opt_state, value=value_opt_state)
    state = state.replace(params=params, opt_state=opt_state)
    return state, metrics


# TODO Add discount, lambda reward_scaling -> `process_data_to_batch`
def init_nn(
    observation_size, action_size, seed=42,
    policy_opt = 'adabelief',
    policy_opt_kwargs = dict(learning_rate=5e-3),
    value_opt = 'adabelief',
    value_opt_kwargs = dict(learning_rate=5e-3),
    policy_kwargs: dict = dict(), value_kwargs: dict = dict(),
):
    rng = jax.random.PRNGKey(seed)
    dummy_action = jnp.zeros((action_size))
    dummy_observation = jnp.zeros((observation_size))
    
    policy_def = lambda x: mlp_categorical(action_size, **policy_kwargs)(x)
    policy_def = hk.transform(policy_def)
    policy_def = hk.without_apply_rng(policy_def)
    policy_opt = getattr(optax, policy_opt)(**policy_opt_kwargs)
    
    value_def = lambda x: mlp_deterministic(1, **value_kwargs)(x).squeeze(-1)
    value_def = hk.transform(value_def)
    value_def = hk.without_apply_rng(value_def)
    value_opt = getattr(optax, value_opt)(**value_opt_kwargs)
    
    rng, rng_policy, rng_value = jax.random.split(rng, 3)
    value_params = value_def.init(rng_policy, dummy_observation)
    value_opt_state = value_opt.init(value_params)
    policy_params = policy_def.init(rng_policy, dummy_observation)
    policy_opt_state = policy_opt.init(policy_params)

    params = {'policy': policy_params, 'value': value_params}
    opt_state = {'policy': policy_opt_state, 'value': value_opt_state}
    
    process_data_to_batch = partial(
        process_data, value_apply=value_def.apply,    
    )
    
    loss = partial(loss_fn, value_apply=value_def.apply, policy_apply=policy_def.apply)
    update = partial(update_fn, 
                     policy_opt=policy_opt.update, 
                     value_opt=value_opt.update,
                     loss_fn=loss)    
    
    
    def make_policy(state):
        fn = lambda rng, x: policy_def.apply(state.params['policy'], x).sample(seed=rng) 
        return jit(fn)
    
    return State(key=rng, params=params, opt_state=opt_state), {
        'process_data': jit(process_data_to_batch),
        'make_policy': make_policy,
        'loss': jit(loss), 'update': jit(update),
        'full_update': jit(lambda state, data: update(state, process_data_to_batch(state.params, data)))
    }


In [2]:
NEnvs = 8
make_env = lambda: gym.make('CartPole-v0')
env = AsyncVectorEnv([make_env for _ in range(NEnvs)])
env_test = gym.make('CartPole-v0')
action_size = env_test.action_space.n
observation_size = env_test.observation_space.shape[0]
state, a2c = init_nn(observation_size, action_size, 42, policy_kwargs={}, value_kwargs={})

In [3]:
def interaction(env, horizon: int = 10, seed: int = 42):
    # -- Initialization
    rng = jax.random.PRNGKey(seed)
    observation, buf = env.reset(), []
    policy = yield
    
    # -- Interaction Loop.
    while True:
        for _ in range(horizon):
            rng, rng_action = jax.random.split(rng)
            action = np.array(policy(rng_action, observation))
            observation_next, reward, done, info = env.step(action)
            buf.append({
                'observation': observation,
                'reward': reward,
                'terminal': 1. - done,
                'action': action
            })
            observation = observation_next.copy()
            
        data = tax.reduce(buf)
        data['last_observation'] = observation
        policy = yield data
        buf = []

In [4]:
interaction_step = interaction(env, 10)
interaction_step.send(None)
policy = a2c['make_policy'](state) 

In [ ]:
for _ in range(1000):
    data = interaction_step.send(policy)
    state, info_fit = a2c['full_update'](state, data)
    policy = a2c['make_policy'](state) 
    interaction_step.send(policy)
    info_eval = evaluation(rng, env_test, policy)
    info = {}
    info.update(**info_fit, **info_eval)
    info = tree.map_structure(lambda v: float(v), info)
    print(info)

In [ ]:
evaluation(rng, env_test, policy)

In [ ]:
policy(rng, env.reset())

In [ ]:
state.params['value']